In [3]:
import setup_django

setup_django.init()

In [4]:
import pandas as pd
from django.conf import settings

ratings_path = settings.DATA_DIR / "ratings_small.csv" #pathlib
ratings_path.exists()

True

In [5]:
df = pd.read_csv(ratings_path)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [7]:
current_users = User.objects.all().values_list('id', flat=True)
rating_users = df['userId'].tolist()

In [11]:
missing_user_ids = set(rating_users) - set(current_users)
missing_user_ids

In [ ]:
for uid in missing_user_ids:
    User.objects.create(
        id=uid,
        username=f"missing-user-{uid}"
    )

In [ ]:
import math
from decimal import Decimal

In [ ]:
df['value'] = df['rating'].apply(lambda x: math.ceil(Decimal(x)))
df.head()

In [ ]:
df['user_id'] = df['userId']
df['object_id'] = df['movieId']

In [ ]:
df.head()

In [ ]:
cols = ['user_id', 'value', 'object_id']
transformed_df = df.copy()[cols]

In [ ]:
rating_records = transformed_df.to_dict('records')

In [ ]:
from ratings.models import Rating
qs = Rating.objects.all()
qs.delete()

In [ ]:
from django.contrib.contenttypes.models import ContentType

ctype = ContentType.objects.get(app_label='movies', model='movie')


In [ ]:
new_ratings = []
for r in rating_records:
    r['content_type'] = ctype
    new_ratings.append(
        Rating(**r)
    )
Rating.objects.bulk_create(new_ratings, ignore_conflicts=True, batch_size=1000)

In [ ]:
from ratings.tasks import task_update_movie_ratings

task_update_movie_ratings()